In [ ]:
import re

import benchlingapi
import Bio.Restriction as Restriction
import pandas as pd
import pygsheets
import toml
from Bio.Seq import Seq
from tqdm.auto import tqdm

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
import paulssonlab.cloning.commands.parser as cmd_parser
import paulssonlab.cloning.commands.semantics as cmd_semantics
import paulssonlab.cloning.enzyme as enzyme
import paulssonlab.cloning.registry as registry
import paulssonlab.cloning.sequence as sequence
import paulssonlab.cloning.util as cloning_util
import paulssonlab.cloning.workflow as workflow
from paulssonlab.api.util import base_url

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
reg = registry.Registry(gc, config["registry"]["folder"])

# Flipped backbone primers

In [ ]:
nonflipped_r = reg.get("oLIB193")["_seq"]
nonflipped_f = reg.get("oLIB194")["_seq"]
uns1 = reg.get("UNS1")["_seq"]
uns10 = reg.get("UNS10")["_seq"]
bb_prefix = reg.get("BioBrick_prefix")["_seq"]
bb_suffix = reg.get("BioBrick_suffix")["_seq"]

In [ ]:
binding_r = nonflipped_r[len(uns1) :]
binding_f = nonflipped_f[len(uns10) :]

In [ ]:
nonflipped_r[: len(uns1)] == uns1.reverse_complement().seq

In [ ]:
nonflipped_f[: len(uns10)] == uns10.seq

In [ ]:
uns1.reverse_complement() + binding_r

In [ ]:
nonflipped_r == sequence.get_seq(uns1.reverse_complement() + binding_r)

In [ ]:
nonflipped_f == sequence.get_seq(uns10 + binding_f)

In [ ]:
uns1.reverse_complement().seq

In [ ]:
uns10.seq

In [ ]:
new_binding_r = Seq("ctctagaagcggccgc")
new_binding_f = Seq("tactagtagcggccgctg")

In [ ]:
new_nonflipped_r = uns1.reverse_complement() + new_binding_r[:-1]
new_nonflipped_f = uns10 + new_binding_f
flipped_r = uns10 + new_binding_r
flipped_f = uns1.reverse_complement() + new_binding_f[:-1]

In [ ]:
primers = {
    "Name": "Bases",
    "oLIB193": nonflipped_r,
    "oLIB194": nonflipped_f,
    "oLIB169": new_nonflipped_r,
    "oLIB170": new_nonflipped_f,
    "oLIB171": flipped_r,
    "oLIB172": flipped_f,
}
for name, seq in primers.items():
    print(f"{name}\t{sequence.get_seq(seq)}")

In [ ]:
# test gibson for 3G

In [ ]:
# single function for syncing a benchling folder of (and optionally attaching primers to sequences?)